In [1]:
import os                                                                        
import time                                                                      
import json
import numpy as np
import pandas as pd
import dask as dd
from timeit import default_timer as timer
import gcsfs, json
from katana import remote
from katana.remote import import_data
from datetime import datetime
from dask.dataframe import from_pandas
from dask.dataframe import to_numeric
# os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

### Transformation

In [2]:
from dask.distributed import Client, progress
client = Client("10.0.0.101:8786")

In [3]:
print(dd.config.get("distributed.client"))

{'heartbeat': '5s', 'scheduler-info-interval': '2s', 'security-loader': None, 'preload': [], 'preload-argv': []}


### Load files into bags

In [4]:
import dask.bag as db
import json
base_path = 'gs://reco-1/all-review-data/'
b = db.read_text(base_path+'All_Amazon_Review.json', blocksize="1024MiB").map(json.loads)
# b.take(1)

In [5]:
# b.count().compute()
#233055327 

### Get Frequencies

In [6]:
frequency=b.map(lambda record: record['reviewerID']).frequencies()

In [7]:
pat2_df=frequency.filter(lambda x : x[1] > 1).to_dataframe()

In [8]:
# at_l_2.take(1)

In [9]:
# pat2=at_l_2.persist()

In [10]:
# pat2.count().compute()

In [11]:
# pat2.npartitions

In [12]:
# pat2_df=pat2.to_dataframe()

In [13]:
# b_df.columns

In [14]:
pat2_df.columns=['reviewerID', 'count']
# pat2_df=pat2_df.persist()
# pat2_df.head()

### Join frequency and the main dataframe

In [15]:
def standardize(record):
    reviewerName=''
    reviewText=''
    summary=''
    vote=''
    unixReviewTime=0
    if 'reviewerName' in record:
        reviewerName = record['reviewerName']
    if 'reviewText' in record:
        reviewText = record['reviewText']
    if 'summary' in record:
        summary = record['summary']
    if 'unixReviewTime' in record:
        unixReviewTime = record['unixReviewTime']
    if 'vote' in record:
        vote = record['vote']

    return {
            'overall': record['overall'],
            'verified': record['verified'],
            'reviewTime': record['reviewTime'],
            'reviewerID': record['reviewerID'],
            'itemId': record['asin'],
            # 'key' : ''.join([record['reviewerID'], '-', record['asin']]),
            'reviewerName': reviewerName,
            'reviewText': reviewText,
            'summary': summary,
            'vote': vote,
            # 'style': record['style']['name']
            'unixReviewTime': unixReviewTime
    }

# o_df=b.map(standardize).to_dataframe()
# write_path = 'gs://reco-1/all-review-data/10-nov-dedup-reviews/at-least-2-reviews-*.csv'
r_final_df=b.map(standardize).to_dataframe().drop_duplicates(subset=['reviewerID','itemId','unixReviewTime'])\
.join(pat2_df.set_index('reviewerID'), on='reviewerID').fillna(0).query('count > 0')\
.astype({'overall':'float', 'verified':'string', 'reviewTime':'string','reviewerID':'string',
                              'itemId':'string','reviewerName':'string', 'reviewText':'string', 'summary':'string', 'unixReviewTime':'int64',
                             'vote':'string'})
#.persist().to_csv(write_path)

In [16]:
# final_df=final_df.persist()

In [17]:
# len(final_df)
# 233M reviews

### Process Items

In [18]:
import dask.bag as db
import json
base_path = 'gs://reco-1/all-review-data/'
m = db.read_text(base_path+'All_Amazon_Meta.json', blocksize="1024MiB").map(json.loads)

In [19]:
def standardize_meta(record):
    description=''
    try:
        description=record['description'][0]
    except:
        description=''
    rank = ''
    price=0.0
    if 'rank' in record:
        try:
            rank=float(str( record['rank']).split(' in')[0].replace(',',''))
        except:
            rank=0.0
    if 'price' in record:
        if len(record['price']) == 0:
            price = 0.0
        else:
            try:
                f1=record['price'].replace('$','')
                price = float(f1)
            except:
                price = 0.0
    
    
    return {
            'description': description,
            'fit': str(record['fit']),
            'title': record['title'],
            'rank': rank,
            'itemId': record['asin'],
            'main_cat' : record['main_cat'],
            'price': price
    }

# o1_df=b.map(standardize).to_dataframe().set_index("key").drop_duplicates(subset=['reviewerID','itemId','unixReviewTime'])
write_path_meta = 'gs://reco-1/all-review-data/10-nov-dedup-reviews/items-*.csv'
final_df=m.map(standardize_meta).to_dataframe().drop_duplicates(subset=['itemId']).astype({'description':'string','fit':'string','title':'string','itemId':'string','main_cat':'string',
                          'rank':'float','price':'float'}).rename(columns={"itemId": "ID"}) 

In [20]:
final_df.dtypes

description     string
fit             string
title           string
rank           float64
ID              string
main_cat        string
price          float64
dtype: object

In [21]:
# len(final_df)
# 14,741,571 items

In [22]:
# write_path_meta = 'gs://reco-1/temp/items-*.csv'


In [23]:
# final_df.to_csv(write_path_meta)

### Store users

In [24]:
import dask.bag as db
import json
base_path = 'gs://reco-1/all-review-data/'
u = db.read_text(base_path+'All_Amazon_Review.json', blocksize="1024MiB").map(json.loads)
# b.take(1)

In [25]:
def standardize_user(record):
    return {
            'reviewerID': record['reviewerID']
    }


u_final_df=u.map(standardize_user).to_dataframe().drop_duplicates(subset=['reviewerID']).rename(columns={"reviewerID": "ID"}) 

In [26]:
# len(final_df)
# #43,531,850 users

In [27]:
# write_path_user = 'gs://reco-1/all-review-data/10-nov-dedup-reviews/users-*.csv'
# final_df.to_csv(write_path_user)

In [28]:
# final_df=final_df

### import

In [29]:
# final_df.npartitions

In [30]:
rc=remote.Client()
rc.server_version

'0.5.0+20221106T212601Z.9c14e060f.dev'

In [31]:
graph = rc.create_graph(num_partitions=7)
graph.graph_id
with import_data.DataFrameImporter(graph) as df_importer:
    df_importer.nodes_dataframe(final_df, id_column="ID", id_space='ITEM', label='ITEM')
    df_importer.nodes_dataframe(u_final_df, id_column="ID", id_space='USER', label='USER')
    # df_importer.nodes_dataframe(av_df, id_column="ID", id_space='ALSO_VIEWED', label='ALSO_VIEWED')
    # df_importer.nodes_dataframe(ab_df, id_column="ID", id_space='ALSO_BOUGHT', label='ALSO_BOUGHT')
    # df_importer.nodes_dataframe(img_df, id_column="ID", id_space='IMAGE', label='IMAGE')
    # df_importer.nodes_dataframe(hrimg_df, id_column="ID", id_space='HIGHRES_IMAGE', label='HIGHRES_IMAGE')
    # df_importer.nodes_dataframe(users_df, id_column="ID", id_space='USER', label='USER')
    # #
    # df_importer.edges_dataframe(av_expl, source_column='also_view', destination_column='asin',source_id_space='ALSO_VIEWED', destination_id_space='ITEM', type='WAS_ALSO_VIEWED')
    # df_importer.edges_dataframe(ab_expl, source_column='also_buy', destination_column='asin',source_id_space='ALSO_BOUGHT', destination_id_space='ITEM', type='WAS_ALSO_BOUGHT')
    # df_importer.edges_dataframe(img_expl, source_column='imageURL', destination_column='asin',source_id_space='IMAGE', destination_id_space='ITEM', type='FOR_ITEM')
    # df_importer.edges_dataframe(hrimg_expl, source_column='imageURLHighRes', destination_column='asin',source_id_space='HIGHRES_IMAGE', destination_id_space='ITEM', type='FOR_ITEM')
    df_importer.edges_dataframe(r_final_df, source_column='reviewerID', destination_column='itemId',source_id_space='USER', destination_id_space='ITEM', type='REVIEWS')
#
    df_importer.insert()

          0/? [?op/s]

          0/? [?op/s]

In [23]:
# graph.delete()

In [32]:
print(f'Graph as {graph.num_nodes()} nodes and {graph.num_edges()} edges')

          0/? [?op/s]

          0/? [?op/s]

Graph as 58273421 nodes and 207760811 edges


In [22]:

qry = """
MATCH(i:ITEM)
where i.ID='0143026860'
return i limit 20
"""
graph.query(qry).view()

          0/? [?op/s]

###### qry = """
MATCH(i:USER)
return i limit 20
"""
graph.query(qry).view()

In [ ]:
qry = """
MATCH(i:ALSO_VIEWED)
return i limit 20
"""
graph.query(qry).view()

In [34]:
qry = """
MATCH(i:ITEM)<-[:REVIEWS]-(a)
where i.ID = 'B000050FDR'
return i,a limit 50
"""
graph.query(qry, contextualize=True).view()

          0/? [?op/s]

          0/? [?op/s]

In [39]:
###needs to be cleaned
qry = """
MATCH(i:ITEM)
return distinct i.main_cat limit 50
"""
graph.query(qry,).view()

          0/? [?op/s]

In [35]:
graph.schema().view()

          0/? [?op/s]

In [37]:
# from katana.remote import export_data
# export_data.rdg(graph, path = "gs://reco-1/rdg/all-data-11-Nov-upd")

          0/? [?op/s]

### Generate Feature

In [ ]:
def udf_pre_processing(g):
    import nltk
    from sklearn.feature_extraction.text import CountVectorizer
    # import blah
    from katana_enterprise.distributed import Graph
    from katana_enterprise.ai.preprocessing.feature_generator import OneHotEncoder
    from katana.ai.preprocessing import BagOfWords
    from katana_enterprise.ai.preprocessing.feature_scaler import StandardScaler
    from katana_enterprise.ai.preprocessing.preprocessing_graph import (
        FeatureDescriptor,
        FeatureType,
        PreprocessingGraph,
    )
    from katana.ai.preprocessing import BagOfWords
    from katana_enterprise.distributed import single_host
    # upsert features
    obj = PreprocessingGraph(g)
    data_dict = {
        "ITEM": [
            FeatureDescriptor(name="main_cat", type=FeatureType.CATEGORICAL, preprocessor_type=OneHotEncoder, impute_function=None),
            FeatureDescriptor(name="price", type=FeatureType.NUMERIC, preprocessor_type=StandardScaler, impute_function=None),
            FeatureDescriptor(name="rank", type=FeatureType.NUMERIC, preprocessor_type=StandardScaler, impute_function=None),
            FeatureDescriptor(name="title", type=FeatureType.STRING, preprocessor_type=BagOfWords)
        ]
    }

    res = obj.encode_features_all(data_dict, target_node_type="ITEM", keep_nan=True)
    print(res.shape)
    print(res.dtype)
    print('finished encoding features')
    obj.upsert_node_feature(feature_name="item_feature", feature_data=res)
    # print('finished upserting features')
    return single_host('ok')


result = graph.run(udf_pre_processing)

### also_view

In [ ]:
av_asin_df = meta_df[['asin', 'also_view']]
len(av_asin_df)

In [ ]:
av_expl=av_asin_df.explode('also_view').dropna()

In [ ]:
av_expl=av_expl.astype({'asin':'string','also_view':'string'})
av_expl.head()

In [ ]:
av_expl.dtypes

#### also_buy

In [ ]:
ab_asin_df = meta_df[['asin', 'also_buy']]
ab_expl=ab_asin_df.explode('also_buy').dropna()
ab_expl=ab_expl.astype({'asin':'string','also_buy':'string'})
ab_expl.head()

#### imageURL

In [ ]:
img_asin_df = meta_df[['asin', 'imageURL']]
img_expl=img_asin_df.explode('imageURL').dropna()
img_expl=img_expl.astype({'asin':'string','imageURL':'string'})
img_expl.head()

#### imageURLHighRes

In [ ]:
hrimg_asin_df = meta_df[['asin', 'imageURLHighRes']]
hrimg_expl=hrimg_asin_df.explode('imageURLHighRes').dropna()
hrimg_expl=hrimg_expl.astype({'asin':'string','imageURLHighRes':'string'})
hrimg_expl.head()

### Process Reviews

In [ ]:
reviews_df.head()

In [ ]:
reviews_df=reviews_df.astype({'overall':'float', 'verified':'string', 'reviewTime':'string','reviewerID':'string',
                              'asin':'string','reviewerName':'string', 'reviewText':'string', 'summary':'string', 'unixReviewTime':'int64',
                             'vote':'string','style':'string','image':'string'})

In [ ]:
reviews_df.compute()

In [ ]:
reviews_df.dtypes

In [ ]:
users_df=reviews_df.reviewerID.unique().to_frame()
users_df=users_df.rename(columns={'reviewerID':'ID'})

In [ ]:
users_df.dtypes

### preprocess graph

In [ ]:
meta_df.dtypes

In [ ]:
meta_final_df=meta_df.drop(columns=['category', 'tech1','description','fit','also_buy','tech2','feature','also_view','details',
                                    'similar_item','date','imageURL','imageURLHighRes', 'price_f2'])  

In [ ]:
meta_final_df.dtypes

In [ ]:
av_df=av_expl.also_view.unique().to_frame()
ab_df=ab_expl.also_buy.unique().to_frame()
img_df=img_expl.imageURL.unique().to_frame()
hrimg_df=hrimg_expl.imageURLHighRes.unique().to_frame()

av_df=av_df.rename(columns={'also_view':'ID'})
ab_df=ab_df.rename(columns={'also_buy':'ID'})
img_df=img_df.rename(columns={'imageURL':'ID'})
hrimg_df=hrimg_df.rename(columns={'imageURLHighRes':'ID'})
#av_df.head()

In [ ]:
#this should not be needed
av_expl=av_expl.repartition(npartitions=1)  
ab_expl=ab_expl.repartition(npartitions=1)
img_expl=img_expl.repartition(npartitions=1)
hrimg_expl=hrimg_expl.repartition(npartitions=1)
# av_expl.compute()
# ab_expl.compute()
# av_expl.head()

In [ ]:
meta_final_df['ID']=meta_final_df['asin']

In [ ]:
meta_final_df.dtypes

### Appendix

In [ ]:
wl=client.get_worker_logs()

In [ ]:
client.get_events("runtimes")

In [ ]:
user_ids=reviews_df.reviewerID.unique()

In [ ]:
len(user_ids)

In [ ]:
fs = gcsfs.GCSFileSystem(project='katana-clusters-beta')
base_path = 'gs://reco-1/beauty-small/'
beautyList = []
print("reading beauty")
with fs.open(base_path+'All_Beauty.json') as f:
    for jsonObj in f:
        beautyDict = json.loads(jsonObj)
        beautyList.append(beautyDict)

print(f'loaded {len(beautyList)} beauty reviews')

In [ ]:
metaBeautyList = []
print("reading beauty meta")
with fs.open(base_path+'meta_All_Beauty.json') as f:
    for jsonObj in f:
        metaBeautyDict = json.loads(jsonObj)
        metaBeautyList.append(metaBeautyDict)

print(f'loaded {len(beautyList)} meta beauty reviews')

In [ ]:
beauty_df = pd.DataFrame.from_dict(beautyList)
meta_beauty_df = pd.DataFrame.from_dict(metaBeautyList)
len(beauty_df)

In [ ]:
beauty_df.head()

In [ ]:
beauty_ddf = from_pandas(beauty_df, npartitions=2)
mb_ddf = from_pandas(meta_beauty_df, npartitions=2)

In [ ]:
mb_ddf.head()

In [ ]:
asins = ddf.asin.unique()

In [ ]:
len(asins)

In [ ]:
class Counter:
    """ A simple class to manage an incrementing counter """
    n = 0

    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1
        return self.n

    def add(self, x):
        self.n += x
        return self.n

from dask.distributed import Client          # Start a Dask Client
client = Client()

future = client.submit(Counter, actor=True)  # Create a Counter on a worker
counter = future.result()                    # Get back a pointer to that object

counter
# <Actor: Counter, key=Counter-1234abcd>

future = counter.increment()                 # Call remote method
future.result()                              # Get back result
# 1

future = counter.add(10)                     # Call remote method
future.result()            

In [ ]:
future = counter.add(10)                     # Call remote method
future.result()        

In [ ]:
beauty_ddf.head()

In [ ]:
beauty_ddf.overall.sum().compute()

### Load Graph

In [ ]:
rc=remote.Client()
rc.server_version


In [ ]:
from katana.remote import import_data
from dask.dataframe import from_pandas
import re


In [ ]:
data = [['tom', '1'], ['nick', '2'], ['juli', '3']]
knows = [['1','2'],['2','3']]
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'ID'])
df=df.astype({'Name': 'str'}, {'ID':'str'})

e_df=pd.DataFrame(knows, columns=['START_ID', 'END_ID'])

names_ddf = from_pandas(df, npartitions=1)
edge_ddf = from_pandas(e_df, npartitions = 1)
print(len(names_ddf))

In [ ]:
names_ddf = names_ddf.astype('string')
edge_ddf = edge_ddf.astype('string')
names_ddf.dtypes

In [ ]:
edge_ddf.dtypes

In [ ]:
names_ddf.head()

In [ ]:
edge_ddf.head()

In [ ]:
graph = rc.create_graph(num_partitions=1)
graph.graph_id
with import_data.DataFrameImporter(graph) as df_importer:
    df_importer.nodes_dataframe(names_ddf, id_column="ID", id_space='PERSON', label='PERSON')
    df_importer.edges_dataframe(edge_ddf, source_column='START_ID', destination_column='END_ID',source_id_space='PERSON', destination_id_space='PERSON', type='KNOWS')

In [ ]:
qry = "match(a) return a"
result = graph.query(qry,contextualize=True)
result.view()